RFM ANALYSIS

In [ ]:
!pip install numpy

In [4]:
!pip install pandas

In [2]:
!pip install seaborn



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv(r"C:\Users\Tanushree Kanakaraj\OneDrive\Desktop\Tanu\Project\consumer360_detailed_retail.csv")
print(df)


       InvoiceNo  CustomerID  ProductID  ProductName     Category    Brand  \
0      INV200000       10037       4759  Product_189      Grocery   Adidas   
1      INV200001       10235       7831  Product_251  Electronics   Loreal   
2      INV200002       10908       4876  Product_268  Accessories   Loreal   
3      INV200003       10072       8934  Product_220      Fashion  Philips   
4      INV200004       10767       4320  Product_240       Beauty   Loreal   
...          ...         ...        ...          ...          ...      ...   
15995  INV215995       10477       4057   Product_14      Fashion     Nike   
15996  INV215996       10556       3060   Product_69      Fashion     Sony   
15997  INV215997       10962       7742   Product_65      Fashion   Adidas   
15998  INV215998       10644       9928  Product_133  Electronics   Adidas   
15999  INV215999       10047       7318  Product_258  Electronics  Philips   

       Quantity  UnitPrice       InvoiceDate Country  Revenue  

In [ ]:
df.head()
df.shape
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    16000 non-null  str    
 1   CustomerID   16000 non-null  int64  
 2   ProductID    16000 non-null  int64  
 3   ProductName  16000 non-null  str    
 4   Category     16000 non-null  str    
 5   Brand        16000 non-null  str    
 6   Quantity     16000 non-null  int64  
 7   UnitPrice    16000 non-null  float64
 8   InvoiceDate  16000 non-null  str    
 9   Country      16000 non-null  str    
 10  Revenue      16000 non-null  float64
dtypes: float64(2), int64(3), str(6)
memory usage: 1.3 MB


In [ ]:
!pip install pyodbc

In [ ]:
!pip install mlxtend

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 1.9 MB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 1.9 MB/s  0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.1 MB 2.6 MB/s eta 0:00:03
   ------- -------------------------------- 1.6/8.1 MB 2.9 MB/s eta 0:00:03
   ------------ --------------------------- 2.6/8.1 MB 3.3 MB/s eta 0:00:02
   ---------------- ----------------------- 3.4/8.1 MB 3.5 MB/s eta 0:00:02
   ---------------------- ----------------- 4.5/8.1 MB 3.7 MB/s eta 0:00:01
   ------------------------- -------------- 5.2/8.

In [6]:
import pandas as pd
import pyodbc
import datetime as dt
from mlxtend.frequent_patterns import apriori, association_rules

In [7]:
conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=Tanushree\SQLEXPRESS;"
    "DATABASE=Consumer360;"
    "Trusted_Connection=yes;"
)

<>:3: DeprecationWarning: invalid escape sequence '\S'
<>:3: DeprecationWarning: invalid escape sequence '\S'
C:\Users\Tanushree Kanakaraj\AppData\Local\Temp\ipykernel_6136\3785896811.py:3: DeprecationWarning: invalid escape sequence '\S'
  "SERVER=Tanushree\SQLEXPRESS;"


In [8]:
df = pd.read_sql("""
SELECT 
    InvoiceNo,
    CustomerID,
    ProductID,
    ProductName,
    Category,
    Brand,
    Quantity,
    UnitPrice,
    InvoiceDate,
    Country,
    Revenue
FROM RetailTransactions
""", conn)

C:\Users\Tanushree Kanakaraj\AppData\Local\Temp\ipykernel_6136\3734254495.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [9]:
print("Data pulled from SQL. Shape:", df.shape)

Data pulled from SQL. Shape: (16000, 11)


In [10]:
import pandas as pd
import datetime as dt

# Convert to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')

# Check conversion
print(df['InvoiceDate'].dtypes)
print(df['InvoiceDate'].head())


datetime64[ns]
0   2023-01-01 00:00:00
1   2023-01-01 01:00:00
2   2023-01-01 02:00:00
3   2023-01-01 03:00:00
4   2023-01-01 04:00:00
Name: InvoiceDate, dtype: datetime64[ns]


In [11]:
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
print("Snapshot date:", snapshot_date)


Snapshot date: 2024-10-13 23:00:00


In [ ]:
rfm = df.groupby(['CustomerID','Country']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  
    'InvoiceNo': 'nunique',                                  
    'Revenue': 'sum'                                         
}).reset_index()

rfm.columns = ['CustomerID','Country','Recency','Frequency','Monetary']

rfm.head()


,CustomerID,Country,Recency,Frequency,Monetary
0,10000,France,613.0,4,6085.20
1,10000,Germany,468.0,3,4527.95
2,10000,India,64.0,3,1971.31
3,10000,UK,NaN,1,916.32
4,10000,USA,561.0,2,555.12


In [13]:
rfm.to_csv(r"C:\Users\Tanushree Kanakaraj\OneDrive\Desktop\Tanu\Project\consumer360_detailed_retail.csv", index=False)
print("RFM segments exported to CSV.")

RFM segments exported to CSV.


MARKET BASKET ANALYSIS


In [26]:
import pandas as pd

print(type(basket))       # should be <class 'pandas.core.frame.DataFrame'>
print(pd.__version__)     # check pandas version


<class 'pandas.DataFrame'>
3.0.0


In [27]:

!pip install --upgrade pandas


In [ ]:
import numpy as np
basket = (basket.values > 0).astype(int)  
basket = pd.DataFrame(basket, index=df.groupby(['InvoiceNo', 'ProductName'])['Quantity'].sum().unstack().fillna(0).index,
                      columns=df.groupby(['InvoiceNo', 'ProductName'])['Quantity'].sum().unstack().fillna(0).columns)

print(basket.head())
print(basket.apply(np.unique))  


ProductName  Product_0  Product_1  Product_10  Product_100  Product_101  \
InvoiceNo                                                                 
INV200000            0          0           0            0            0   
INV200001            0          0           0            0            0   
INV200002            0          0           0            0            0   
INV200003            0          0           0            0            0   
INV200004            0          0           0            0            0   

ProductName  Product_102  Product_103  Product_104  Product_105  Product_106  \
InvoiceNo                                                                      
INV200000              0            0            0            0            0   
INV200001              0            0            0            0            0   
INV200002              0            0            0            0            0   
INV200003              0            0            0            0           

In [31]:
print("Basket shape:", basket.shape)
print("Non-zero entries per column:")
print((basket > 0).sum().sort_values(ascending=False).head(10))  # top 10 popular products


Basket shape: (16000, 300)
Non-zero entries per column:
ProductName
Product_38     80
Product_182    75
Product_70     74
Product_200    71
Product_141    68
Product_246    68
Product_137    68
Product_279    67
Product_219    67
Product_173    66
dtype: int64


In [32]:
from mlxtend.frequent_patterns import apriori, association_rules

# Try a smaller support, e.g., 0.01 (1%) or even 0.005
freq_items = apriori(basket, min_support=0.01, use_colnames=True)

print("Number of frequent itemsets found:", len(freq_items))


Number of frequent itemsets found: 0


c:\Users\Tanushree Kanakaraj\OneDrive\Desktop\Tanu\Project\.venv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:175: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [33]:
if not freq_items.empty:
    rules = association_rules(freq_items, metric='lift', min_threshold=1)
    rules.to_csv(r"C:\Users\Tanushree Kanakaraj\OneDrive\Desktop\Tanu\Project\market_basket_rules.csv", index=False)
    print("Market basket rules exported. Total rules:", len(rules))
else:
    print("No frequent itemsets found. Try lowering min_support.")


No frequent itemsets found. Try lowering min_support.


In [35]:
popular_products = basket.columns[(basket > 0).sum() >= 5]  # products appearing in ≥10 invoices
basket_filtered = basket[popular_products]
print(basket_filtered)

ProductName  Product_0  Product_1  Product_10  Product_100  Product_101  \
InvoiceNo                                                                 
INV200000            0          0           0            0            0   
INV200001            0          0           0            0            0   
INV200002            0          0           0            0            0   
INV200003            0          0           0            0            0   
INV200004            0          0           0            0            0   
...                ...        ...         ...          ...          ...   
INV215995            0          0           0            0            0   
INV215996            0          0           0            0            0   
INV215997            0          0           0            0            0   
INV215998            0          0           0            0            0   
INV215999            0          0           0            0            0   

ProductName  Product_102

In [15]:
rfm.to_csv(r"C:\Users\Tanushree Kanakaraj\OneDrive\Desktop\Tanu\Project\consumer360_detailed_retail.csv", index=False)
print("RFM segments exported to CSV.")


RFM segments exported to CSV.
